In [127]:
import pandas as pd
import random

# Load the data
moderator_data = pd.read_excel("./Scoring/moderator_scored.xlsx")

# Moderator Matching Algorithm
**Purpose:**
- Increase Revenue: By matching the best reviewers with the top priority ads, we ensure that high-quality ads get the attention they deserve, leading to better user experience and potentially higher revenue

- Improve Resource Utilization: By dynamically allocating tasks based on moderator capacity and expertise, we aim to increase the utilization of ad moderators by at most 10%

In [128]:
# Extract unique markets from the moderator data
all_markets = set()
for market_list in moderator_data['market']:
    markets = eval(market_list)
    all_markets.update(markets)

# Generate 5000 sample ads with random queue_market attributes and random ad scores
sample_ads_50 = []
for _ in range(500):
    ad = {
        'ad_id': f"ad_{_ + 1}",
        'queue_market': random.choice(list(all_markets)),
        'ad_score': random.random(),  # Random score between 0 and 1
        'confidence': random.random()
    }
    sample_ads_50.append(ad)

# Gurobipy Optimization Model
1. **Decision Variables**
- For each ad and each moderator, a binary decision variable is created to represent whether the ad is assigned to the moderator (1) or not (0)

2. **Objective Function**
- The first objective function to minimize the difference between ad's score and moderator's score, this ensures that the top ads get matched with the best moderators
- The second objective function is with regards to the confidence of the ad, which is defined as how confident our ___ model is in identifying whether there is a possible violation in the ad
    - If confidence is high, the ad is "easier" to moderator as it is easy to spot whether there is a violation. Hence the ad will be allocated to moderators that have high productivity so the ad can be cleared quickly, and low accuracy as a tradeoff
    - For ads with low confidence, it will be allocated to moderators that have high accuracy so that violations can be correctly identified, with the tradeoff being low productivity
- Combining these two objective functions, the overall objective function is as follows:
$$
\text{Minimize: } \sum_{\text{ad, mod}} \left| \beta_1 (ad\_score - moderator\_score) + \beta_2 (ad\_confidence - normalized\_productivity + normalized\_accuracy) \right|
$$

The coefficients $\beta_1$ and $\beta_2$ in the objective function represent weights that determine the importance of matching `ad_score` with `moderator_score` and `ad_confidence` with `normalized_productivity` and `normalized_accuracy`, respectively. Ideally, these coefficients should be derived from an iterative training and optimization process of the Gurobi model. By analyzing the results over multiple iterations and adjusting these coefficients, one can refine the model's performance and achieve better task allocations that meet specific business objectives.

However, due to constraints in our current setup, we are unable to conduct this iterative training. As a result, for the purpose of this demonstration, we are using placeholder values for $\beta_1$ and $\beta_2$, both assumed to be 0.5. This means that, in our current model, both parts of the objective function are given equal importance. 

3. **Constraints**
- One ad should only be allocated 1 moderator
- The total tasks allocated to each moderator cannot exceed their max_tasks_per_day
- The delivery_country of the ad must match the market of the moderator
- If moderator's category expertise is not null, expertise must match category of ad
- If null, each moderator can only be assigned a maximum of 3 ad categories a day
    - This is to ensure that the moderator's work is more focused to improve productivity

In [129]:
from gurobipy import Model, GRB

# Initialize the Gurobi model
m = Model("AdTaskAllocation")

# Create the decision variables
x = {}
for ad in sample_ads_50:
    for _, mod_row in moderator_data.iterrows():
        mod = mod_row['moderator']
        x[ad['ad_id'], mod] = m.addVar(vtype=GRB.BINARY, name=f"x_{ad['ad_id']}_{mod}")

In [122]:
# Set the objective function
m.setObjective(sum(x[ad['ad_id'], mod] * abs(0.5 * (ad['ad_score'] - mod_row['moderator_score']) + 
    0.5 * (ad['confidence'] - mod_row['normalized_productivity'] + mod_row['normalized_accuracy'])) 
                for ad in sample_ads_50 for _, mod_row in moderator_data.iterrows()), GRB.MINIMIZE)

In [123]:
# Add the constraints

# TODO: Add constraints 4 and 5

# Each ad should be allocated to only one moderator
for ad in sample_ads_50:
    m.addConstr(sum(x[ad['ad_id'], mod_row['moderator']] for _, mod_row in moderator_data.iterrows()) == 1)

# The total tasks assigned to a moderator should not exceed their max tasks per day
for _, mod_row in moderator_data.iterrows():
    mod = mod_row['moderator']
    m.addConstr(sum(x[ad['ad_id'], mod_row['moderator']] for ad in sample_ads_50) <= mod_row['max_tasks_per_day'])

# Only assign an ad to a moderator if the ad's market matches the moderator's market
for ad in sample_ads_50:
    for _, mod_row in moderator_data.iterrows():
        mod = mod_row['moderator']
        if ad['queue_market'] not in eval(mod_row['market']):
            m.addConstr(x[ad['ad_id'], mod] == 0)

In [124]:
# Solve the model
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 622370 rows, 642500 columns and 1905585 nonzeros
Model fingerprint: 0x5473690a
Variable types: 0 continuous, 642500 integer (642500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 4e+04]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.24 seconds (0.07 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [125]:
# Extract the assignments from the solution
assignments = {}
for ad in sample_ads_50:
    for _, mod_row in moderator_data.iterrows():
        if x[ad['ad_id'], mod_row['moderator']].x > 0.5:  # If this ad is assigned to this moderator
            assignments[ad['ad_id']] = mod_row['moderator']

assignments

{'ad_1': 1739288436103169,
 'ad_2': 2287729,
 'ad_3': 1752804614390817,
 'ad_4': 1768203847936033,
 'ad_5': 9152229,
 'ad_6': 2803125,
 'ad_7': 1714960827917314,
 'ad_8': 1738944757507074,
 'ad_9': 1723656806883329,
 'ad_10': 5869200,
 'ad_11': 1671869410894854,
 'ad_12': 7050108,
 'ad_13': 2662691,
 'ad_14': 1690600005280769,
 'ad_15': 6853731,
 'ad_16': 7989612,
 'ad_17': 1738944494259201,
 'ad_18': 1711506485296130,
 'ad_19': 1724470463815682,
 'ad_20': 7677885,
 'ad_21': 8085619,
 'ad_22': 8959015,
 'ad_23': 9630899,
 'ad_24': 2963527,
 'ad_25': 1690239244530689,
 'ad_26': 1715218806419458,
 'ad_27': 1743522632016930,
 'ad_28': 3318739,
 'ad_29': 3571359,
 'ad_30': 1672987155331077,
 'ad_31': 1689841489642498,
 'ad_32': 2927720,
 'ad_33': 1700711913253889,
 'ad_34': 3026838,
 'ad_35': 1716603082926081,
 'ad_36': 3857259,
 'ad_37': 1686755036370945,
 'ad_38': 1672187707661318,
 'ad_39': 5323368,
 'ad_40': 1754816678742065,
 'ad_41': 3318739,
 'ad_42': 1736956048423938,
 'ad_43': 175

In [126]:
# Convert the dictionary of assignments into a DataFrame
assignments_df = pd.DataFrame(list(assignments.items()), columns=['ad_id', 'moderator'])

# Count the number of ads assigned to each moderator
assigned_counts = assignments_df.groupby('moderator').size().reset_index(name='num_ads_assigned')

# Merge with the original moderator_data
merged_data = pd.merge(moderator_data, assigned_counts, left_on='moderator', right_on='moderator', how='left')

# Fill NaN values with 0 (for moderators with no assignments)
merged_data['num_ads_assigned'].fillna(0, inplace=True)

# Calculate the increase in utilization % and new utilization % for each moderator
merged_data['increase_in_utilisation'] = (merged_data['num_ads_assigned'] * merged_data['handling time']) / (PAID_HOURS_PER_DAY * 60 * 60 * 1000)
merged_data['new_utilisation'] = merged_data['Utilisation %'] + merged_data['increase_in_utilisation']

# Extract the relevant columns for display
output_table = merged_data[['moderator', 'num_ads_assigned', 'increase_in_utilisation', 'new_utilisation', 'max_tasks_per_day']].sort_values(['new_utilisation'])

output_table = output_table[output_table['num_ads_assigned'] > 0]

output_table

,moderator,num_ads_assigned,increase_in_utilisation,new_utilisation,max_tasks_per_day
1282,7167613,2.0,0.000007,0.000007,29090.909091
1283,9020538,1.0,0.001059,0.001059,94.466494
1279,1691911660815362,1.0,0.001312,0.001312,76.210638
1278,6092503,4.0,0.000025,0.006338,16000.000000
1272,1735599208242177,2.0,0.000928,0.049594,215.584999
...,...,...,...,...,...
22,1764066885924882,2.0,0.008630,1.007943,23.174412
16,1695311101589506,1.0,0.000004,1.045108,27692.307692
11,1752090693569553,2.0,0.007795,1.075782,25.659073
5,1695096148334594,1.0,0.002646,1.118160,37.795772
